To run this you need an specific enviroment with the packages installed:

In [1]:
from skopt import BayesSearchCV
from xgboost import XGBClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [2]:
# Example dataset (replace with your own data)
data = load_iris()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
# Create an XGBoost classifier
xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Define the hyperparameter search space
search_space = {
    'n_estimators': (100, 1000),        # range of values to search
    'max_depth': (3, 10),               # range of values to search
    'learning_rate': (0.01, 0.3, 'log-uniform'),  # log-uniform distribution
    'subsample': (0.5, 1.0),            # range of values to search
    'colsample_bytree': (0.5, 1.0)      # range of values to search
}


In [5]:
# Initialize BayesSearchCV
bayes_search = BayesSearchCV(
    estimator=xgb,
    search_spaces=search_space,
    n_iter=50,   # Number of parameter settings that are sampled
    cv=3,        # Cross-validation splitting strategy
    n_jobs=-1,   # Use all available cores
    random_state=42,
    verbose= 2
)

# Perform the search
bayes_search.fit(X_train, y_train)


Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END colsample_bytree=0.705051979426657, learning_rate=0.11883357504897696, max_depth=10, n_estimators=384, subsample=0.8350739741344673; total time=   0.0s
[CV] END colsample_bytree=0.705051979426657, learning_rate=0.11883357504897696, max_depth=10, n_estimators=384, subsample=0.8350739741344673; total time=   0.1s
[CV] END colsample_bytree=0.705051979426657, learning_rate=0.11883357504897696, max_depth=10, n_estimators=384, subsample=0.8350739741344673; total time=   0.1s
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END colsample_bytree=0.9186941777766422, learning_rate=0.2017268796559709, max_depth=5, n_estimators=956, subsample=0.9320639577324754; total time=   0.1s
[CV] END colsample_bytree=0.9186941777766422, learning_rate=0.2017268796559709, max_depth=5, n_estimators=956, subsample=0.9320639577324754; total time=   0.1s
[CV] END colsample_bytree=0.9186941777766422, learning_rate=0.2017268796559709,

BayesSearchCV(cv=3,
              estimator=XGBClassifier(base_score=None, booster=None,
                                      callbacks=None, colsample_bylevel=None,
                                      colsample_bynode=None,
                                      colsample_bytree=None, device=None,
                                      early_stopping_rounds=None,
                                      enable_categorical=False,
                                      eval_metric='mlogloss',
                                      feature_types=None, gamma=None,
                                      grow_policy=None, importance_type=None,
                                      interaction_constraints=None,
                                      learning_rat...
                                      max_leaves=None, min_child_weight=None,
                                      missing=nan, monotone_constraints=None,
                                      multi_strategy=None, n_estimators=None,
                                      n_jobs=None, num_parallel_tree=None,
                                      random_state=None, ...),
              n_jobs=-1, random_state=42,
              search_spaces={'colsample_bytree': (0.5, 1.0),
                             'learning_rate': (0.01, 0.3, 'log-uniform'),
                             'max_depth': (3, 10), 'n_estimators': (100, 1000),
                             'subsample': (0.5, 1.0)},
              verbose=2)

In [17]:
import pandas as pd
best_param = bayes_search.best_params_
best_score = bayes_search.best_score_

print(f"Best parameters: {best_param}")
print(f"Best score: {best_score}")

Best parameters: OrderedDict([('colsample_bytree', 1.0), ('learning_rate', 0.01), ('max_depth', 3), ('n_estimators', 100), ('subsample', 0.5312099442549354)])
Best score: 0.9583333333333334


In [18]:
best_param["best_score"] = best_score  
print(best_param)

OrderedDict([('colsample_bytree', 1.0), ('learning_rate', 0.01), ('max_depth', 3), ('n_estimators', 100), ('subsample', 0.5312099442549354), ('best_score', 0.9583333333333334)])


In [23]:
param_best = bayes_search.best_params_
param_best

OrderedDict([('colsample_bytree', 1.0),
             ('learning_rate', 0.01),
             ('max_depth', 3),
             ('n_estimators', 100),
             ('subsample', 0.5312099442549354),
             ('best_score', 0.9583333333333334),
             ('trial', 3)])

In [22]:
a = 3
dict_trial = param_best

dict_trial["trial"] = a

print(dict_trial)
print(param_best)

OrderedDict([('colsample_bytree', 1.0), ('learning_rate', 0.01), ('max_depth', 3), ('n_estimators', 100), ('subsample', 0.5312099442549354), ('best_score', 0.9583333333333334), ('trial', 3)])
OrderedDict([('colsample_bytree', 1.0), ('learning_rate', 0.01), ('max_depth', 3), ('n_estimators', 100), ('subsample', 0.5312099442549354), ('best_score', 0.9583333333333334), ('trial', 3)])


In [21]:
best_model = bayes_search.best_estimator_ #Here the best model is saved
best_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [2]:
import sys
import os

# Get the project directory
project_path = "/home/diego-ngz/Git/thesis-tabtrans"

sys.path.append(project_path) #This helps to be able to import the data from the parent directory to other files

from utils import data

In [16]:
import time
import cupy as cp
from cuml.model_selection import train_test_split
from sklearn.datasets import fetch_covtype
import xgboost as xgb
from sklearn.model_selection import GridSearchCV


In [53]:
task_id = 31
sample = 100

X_train, X_test, y_train, y_test, train_indices, val_indices, n_instances, n_labels, n_numerical, n_categories = data.import_data(df_id)
type(X_train)

numpy.ndarray

In [54]:
X_train = cp.array(X_train)
X_test = cp.array(X_test)
y_train = cp.array(y_train)
y_test = cp.array(y_test)

In [55]:
type(X_train)
print(X_train.shape)
X_val_test = X_train[val_indices]


(800, 20)
(160, 20)


In [60]:
from sklearn.metrics import accuracy_score



# Define the parameter grid
param_grid = {
    'max_depth': [2, 3, 4],
    'learning_rate': [0.01, 0.1, 1],
    'n_estimators': [100, 300, 500],
}

scores = []
config_num = 1

# Iterate over all combinations of hyperparameters
for max_depth in param_grid['max_depth']:
    for learning_rate in param_grid['learning_rate']:
        for n_estimators in param_grid['n_estimators']:
            
            # Create the model with current hyperparameters
            bst = xgb.XGBClassifier(
                n_estimators=n_estimators,
                max_depth=max_depth,
                learning_rate=learning_rate,
                objective='multi:softmax',
                num_class = n_labels,
                device='cpu'
            )

            # Convert CuPy arrays to NumPy for fitting
            bst.fit(X_train[train_indices],y_train[train_indices]),
            # Make predictions
            y_pred = bst.predict(X_train[val_indices])

            # Evaluate the model
            score = accuracy_score(cp.asnumpy(y_train[val_indices]), y_pred)
            scores.append(score)





/home/diego-ngz/anaconda3/envs/xgboost-gpu/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [20:40:43] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1706051839871/work/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cpu, while the input data is on: cuda:0.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [61]:
len(scores)

27

In [48]:
bst = xgb.XGBClassifier(n_estimators=3000, max_depth=2, learning_rate=1,num_class = n_labels, objective='multi:softmax', device= 'cuda')


In [49]:
# fit model
bst.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=3000, n_jobs=None, num_class=2,
              num_parallel_tree=None, ...)

In [50]:
bst2.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cpu', early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=3000, n_jobs=None, num_class=2,
              num_parallel_tree=None, ...)

In [35]:
# make predictions
preds = bst.predict(X_test)

In [36]:
preds

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0], dtype=int32)

In [13]:
param_grid = {
    'learning_rate': [0.01, 0.05]
}

In [14]:
clf_xgb = xgb.XGBClassifier(objective='multi:softmax', num_class = n_labels, seed = 11, device= 'cuda')


In [20]:
from hyperopt import tpe, STATUS_OK, Trials, hp, fmin, STATUS_OK, space_eval

In [25]:
# Define the hyperparameter space
space = {
    'max_depth': hp.quniform('max_depth', 2, 8, 1),
    'learning_rate': hp.loguniform('learning_rate', -5, -2),
    'subsample': hp.uniform('subsample', 0.5, 1)
}

In [24]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

# Set up the k-fold cross-validation
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

In [29]:
# Objective function
def objective(params):
    
    xgboost = xgb.XGBClassifier(**params,objective='multi:softmax', num_class = n_labels, seed = 11, device= 'cuda')
    score = cross_val_score(estimator=xgboost, 
                            X=X_train, 
                            y=y_train, 
                            cv=kfold, 
                            scoring='recall', 
                            n_jobs=-1).mean()

    # Loss is negative score
    loss = - score

    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

In [18]:

# Define the hyperparameter space
space = {
    'learning_rate': [0.01, 0.05],
    'n_estimators': [50, 100]
}

# Define the objective function to minimize
def objective(params):
    xgb_model = xgb.XGBClassifier(**params)
    xgb_model.fit(X_train, y_train)
    y_pred = xgb_model.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    return {'loss': -score, 'status': STATUS_OK}

# Perform the optimization
best_params = fmin(objective, space, algo=tpe.suggest, max_evals=100)
print("Best set of hyperparameters: ", best_params)

In [19]:
# Define Parameters

num_round = 3000

clf = xgb.XGBClassifier(device="cuda", n_estimators=num_round)

In [ ]:
# Leave most parameters as default
clf = xgb.XGBClassifier(device="cuda", n_estimators=num_round)
# Train model
start = time.time()
clf.fit(X_train, y_train, eval_set=[(X_test, y_test)])
gpu_res = clf.evals_result()
print("GPU Training Time: %s seconds" % (str(time.time() - start)))